Introduction:

A pizza place in Columbus Ohio is looking to expand into other nearby cities in Columbus.  They know that their most successful pizza location in Columbus is near parks.  There seems to be an increase in sales when the pizza store is close to parks.  They are now looking for which nearby city in Franklin County doesn't not have a pizza place as their top venue, and has parks in their top 5 venues.  The goal is to build the new pizza place in a city in Franklin county that has parks and does not have pizza places as their top venue.  

Data needed:
    
We are going to need a list of the zip codes in Ohio filtered to those in Franklin County.  The latitude and longitude of the other cities in Franklin County will need to be calculated so that a list of venues can be found using the foursquare api.

Methodology:

We will need to look at the frequency of venues in those zip codes and filter out any that have pizza places as their most common venue along with the City of Columbus as they already have a location there.  Next we will look for locations that have parks as one of their top 5 venues.  Clustering will be used to compare the different Franklin County cities and find an ideal location where there is a high frequency of parks with low frequency of other Pizza places.  

In [20]:
#import libraries
import pandas as pd
import numpy as np 
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import requests
print('all-loaded')

all-loaded


In [31]:
# create df from ohio zip code list
df = pd.read_csv('C:/Users/micha/OneDrive/Documents/IBM_DS/Capstone/Ohio_zip.csv', dtype={'Zip Code':str})
df.head()
df.tail()

,Zip Code,City,County
1442,45896,Waynesfield,Auglaize
1443,45897,Williamstown,Hancock
1444,45898,Willshire,Van Wert
1445,45899,Wren,Van Wert
1446,45999,Cincinnati,Hamilton


In [32]:
df.shape

(1447, 3)

In [45]:
#Test with single OH Zip
from geopy.geocoders import Nominatim
address = 'Columbus, Ohio 43215'

geolocator = Nominatim(user_agent="oh_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of OhioZip are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of OhioZip are 39.9622601, -83.0007065.


In [34]:
#filter to just the Cities in Franklin County
cbus = df[df['County'].str.contains('Franklin')].reset_index(drop=True)
cbus.head()

,Zip Code,City,County
0,43002,Amlin,Franklin
1,43004,Blacklick,Franklin
2,43016,Dublin,Franklin
3,43017,Dublin,Franklin
4,43026,Hilliard,Franklin


In [35]:
#left with 66 cities
cbus.shape

(66, 3)

In [36]:
#use geolocator to get the lat/Long associated with each zip code
lat = []
long = []
zipc = cbus['City'] + ", Ohio " + cbus['Zip Code']
for i in zipc:
    if geolocator.geocode(i) != None:
        cp = geolocator.geocode(i)
        latr = cp.latitude
        lonr = cp.longitude
        lat.append(latr)
        long.append(lonr)
        print(latr) 
        print(lonr)
    else: 
        lat.append('NA') 
        long.append('NA')

40.076401
-83.1807987
39.9956195
-82.811286
40.0992294
-83.1140771
40.0992294
-83.1140771
40.033814
-83.1596108
40.0811745
-82.8087864
39.9547861
-82.8121191
40.126139
-82.9295287
40.0930945
-83.0179593
40.126139
-82.9295287
39.91700175
-82.83114343049847
39.842920500000005
-82.81594617222837
39.9139207
-83.1633336
39.8814519
-83.0929645
39.8522449
-82.8868636
39.810827599999996
-83.16823847278656
39.8092301
-82.972404
39.8522449
-82.8868636
39.8522449
-82.8868636
39.8522449
-82.8868636
39.8522449
-82.8868636
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9667308
-82.8854559
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9622601
-83.0007065
39.9945084
-83.0624078
39.9622601
-83.0007065
39

In [37]:
print(lat)
print(long)

[40.076401, 39.9956195, 40.0992294, 40.0992294, 40.033814, 40.0811745, 39.9547861, 40.126139, 40.0930945, 40.126139, 39.91700175, 39.842920500000005, 39.9139207, 39.8814519, 39.8522449, 39.810827599999996, 39.8092301, 39.8522449, 39.8522449, 39.8522449, 39.8522449, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9667308, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9945084, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 40.0195625, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601, 39.9622601]
[-83.1807987, -82.811286, -83.1140771, -83.1140771, -83.1596108, -82.8087864, -82.8121191, -82.9295287, -83.0179593, -82.9295287, -82.83114343049847, -82.81594617222837, -83.1633336, -83.0929645

In [38]:
#add the lat and long to the cbus dataframe
cbus['Latitude'] = lat
cbus['Longitude'] = long
cbus = cbus.rename(columns={'Zip Code':'ZipCode'})
cbus.head()

,ZipCode,City,County,Latitude,Longitude
0,43002,Amlin,Franklin,40.076401,-83.180799
1,43004,Blacklick,Franklin,39.995619,-82.811286
2,43016,Dublin,Franklin,40.099229,-83.114077
3,43017,Dublin,Franklin,40.099229,-83.114077
4,43026,Hilliard,Franklin,40.033814,-83.159611


In [39]:
cbus.shape

(66, 5)

In [46]:
#plotted all the location first
map_cbus = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, zipcode in zip(cbus['Latitude'], cbus['Longitude'], cbus['City'], cbus['ZipCode']):
    label = '{}, {}'.format(city, zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cbus)  
    
map_cbus

In [47]:
# set all the foursquare credentials and version
CLIENT_ID = 'OS1NCB3H0CRPW2ZH2RDGOXX4VLZCKHYQGYO0MO3BURFKWRWG' # your Foursquare ID
CLIENT_SECRET = 'HLAX5PI4M5DR1BAXQLLDVLLS4O4IOPNQDKS00VQD34OIOGZI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OS1NCB3H0CRPW2ZH2RDGOXX4VLZCKHYQGYO0MO3BURFKWRWG
CLIENT_SECRET:HLAX5PI4M5DR1BAXQLLDVLLS4O4IOPNQDKS00VQD34OIOGZI


In [52]:
#look at the first city in the df
cbus_latitude = cbus.loc[0, 'Latitude'] # City latitude value
cbus_longitude = cbus.loc[0, 'Longitude'] # City longitude value

city_name = cbus.loc[0, 'City'] # City name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               cbus_latitude, 
                                                               cbus_longitude))

Latitude and longitude values of Amlin are 40.076401, -83.1807987.


In [93]:
LIMIT = 1500 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    cbus_latitude, 
    cbus_longitude, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600dd92930d8072f432e7f21'},
 'response': {'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 40.0809010045, 'lng': -83.17492875630963},
   'sw': {'lat': 40.071900995499995, 'lng': -83.18666864369037}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f32396b19836c91c7c268e4',
       'name': "Mama Bucci's Pizza",
       'location': {'address': '6978 Rings Rd',
        'lat': 40.07691955566406,
        'lng': -83.18052673339844,
        'labeledLatLngs': [{'label': 'entrance',
          'lat': 40.076972,
          'lng': -83.180744},
         {'label': 'display',
          'lat': 40.07691955566406,
          

In [110]:
# function for all neighborhoods in Franklin County
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [58]:
#run function for the toronto venues/hoodz
cbus_venues = getNearbyVenues(names=cbus['City'],
                                   latitudes=cbus['Latitude'],
                                   longitudes=cbus['Longitude']
                                  )

Amlin
Blacklick
Dublin
Dublin
Hilliard
New Albany
Reynoldsburg
Westerville
Worthington
Westerville
Brice
Canal Winchester
Galloway
Grove City
Groveport
Harrisburg
Lockbourne
Groveport
Groveport
Groveport
Groveport
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Whitehall
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Upper Arlington
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Gahanna
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus
Columbus


In [95]:
#check out the data frame
print(cbus_venues.shape)
cbus_venues.head()

(2357, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amlin,40.076401,-83.180799,Mama Bucci's Pizza,40.076920,-83.180527,Pizza Place
1,Amlin,40.076401,-83.180799,NiDovi Pizza & BBQ,40.076947,-83.180631,BBQ Joint
2,Amlin,40.076401,-83.180799,Amlin Crossing Park,40.076557,-83.185007,Park
3,Amlin,40.076401,-83.180799,Chapman Drainage & Basement Repair,40.079369,-83.182673,Home Service
4,Blacklick,39.995619,-82.811286,Train Stop,39.996730,-82.810660,Light Rail Station


In [96]:
# how many venues per city
cbus_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Amlin,4,4,4,4,4,4
Blacklick,4,4,4,4,4,4
Brice,4,4,4,4,4,4
Canal Winchester,1,1,1,1,1,1
Columbus,2058,2058,2058,2058,2058,2058
Dublin,52,52,52,52,52,52
Gahanna,33,33,33,33,33,33
Galloway,4,4,4,4,4,4
Grove City,11,11,11,11,11,11


In [97]:
# one hot encoding
cbus_onehot = pd.get_dummies(cbus_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
cbus_onehot['City'] = cbus_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [cbus_onehot.columns[-1]] + list(cbus_onehot.columns[:-1])
cbus_onehot = cbus_onehot[fixed_columns]

cbus_onehot.shape

(2357, 114)

In [98]:
#get frequency of occurrence
cbus_grouped = cbus_onehot.groupby('City').mean().reset_index()
cbus_grouped

,City,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,...,Theater,Tourist Information Center,Trail,Video Game Store,Video Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Yoga Studio
0,Amlin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Blacklick,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Brice,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Canal Winchester,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Columbus,0.040816,0.000000,0.020408,0.020408,0.020408,0.000000,0.000000,0.000000,0.000000,...,0.061224,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000
5,Dublin,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462
6,Gahanna,0.030303,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.030303,...,0.000000,0.030303,0.000000,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000
7,Galloway,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Grove City,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,...,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Groveport,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [99]:
# take top 10 per hood
num_top_venues = 10

for hood in cbus_grouped['City']:
    print("----"+city+"----")
    temp = cbus_grouped[cbus_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Columbus----
                     venue  freq
0             Home Service  0.25
1                BBQ Joint  0.25
2              Pizza Place  0.25
3                     Park  0.25
4      American Restaurant  0.00
5  New American Restaurant  0.00
6              Record Shop  0.00
7               Public Art  0.00
8                      Pub  0.00
9              Post Office  0.00


----Columbus----
                 venue  freq
0        Garden Center  0.50
1     Business Service  0.25
2   Light Rail Station  0.25
3  American Restaurant  0.00
4          Record Shop  0.00
5           Public Art  0.00
6                  Pub  0.00
7          Post Office  0.00
8                 Pool  0.00
9                Plaza  0.00


----Columbus----
            venue  freq
0  Hardware Store  0.25
1   Grocery Store  0.25
2  Discount Store  0.25
3     Pizza Place  0.25
4    Neighborhood  0.00
5      Public Art  0.00
6             Pub  0.00
7     Post Office  0.00
8            Pool  0.00
9           Plaza  0.00

In [100]:
#function to sort them in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [101]:
#look at the top 10 venues per city
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['City'] = cbus_grouped['City']

for ind in np.arange(cbus_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cbus_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amlin,Home Service,BBQ Joint,Pizza Place,Park,American Restaurant,New American Restaurant,Record Shop,Public Art,Pub,Post Office
1,Blacklick,Garden Center,Business Service,Light Rail Station,American Restaurant,Record Shop,Public Art,Pub,Post Office,Pool,Plaza
2,Brice,Hardware Store,Grocery Store,Discount Store,Pizza Place,Neighborhood,Public Art,Pub,Post Office,Pool,Plaza
3,Canal Winchester,Skate Park,American Restaurant,Rental Car Location,Public Art,Pub,Post Office,Pool,Plaza,Playground,Pizza Place
4,Columbus,Park,Theater,Hotel,Coffee Shop,American Restaurant,Café,Gym / Fitness Center,Asian Restaurant,Cuban Restaurant,Spa


In [102]:
#as mentioned in the intro, they want to build somewhere outside of columbus so we drop columbus from the dataframe
wo_cbus = neighborhoods_venues_sorted[neighborhoods_venues_sorted['City']!= 'Columbus']
wo_cbus.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amlin,Home Service,BBQ Joint,Pizza Place,Park,American Restaurant,New American Restaurant,Record Shop,Public Art,Pub,Post Office
1,Blacklick,Garden Center,Business Service,Light Rail Station,American Restaurant,Record Shop,Public Art,Pub,Post Office,Pool,Plaza
2,Brice,Hardware Store,Grocery Store,Discount Store,Pizza Place,Neighborhood,Public Art,Pub,Post Office,Pool,Plaza
3,Canal Winchester,Skate Park,American Restaurant,Rental Car Location,Public Art,Pub,Post Office,Pool,Plaza,Playground,Pizza Place
5,Dublin,Pizza Place,Italian Restaurant,Bar,Convention Center,Gift Shop,Hobby Shop,Ice Cream Shop,Irish Pub,Neighborhood,American Restaurant


In [103]:
#drop locations where the most common venue is a Pizza Place
wo_pz = wo_cbus[wo_cbus['1st Most Common Venue']!= 'Pizza Place']
wo_pz.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amlin,Home Service,BBQ Joint,Pizza Place,Park,American Restaurant,New American Restaurant,Record Shop,Public Art,Pub,Post Office
1,Blacklick,Garden Center,Business Service,Light Rail Station,American Restaurant,Record Shop,Public Art,Pub,Post Office,Pool,Plaza
2,Brice,Hardware Store,Grocery Store,Discount Store,Pizza Place,Neighborhood,Public Art,Pub,Post Office,Pool,Plaza
3,Canal Winchester,Skate Park,American Restaurant,Rental Car Location,Public Art,Pub,Post Office,Pool,Plaza,Playground,Pizza Place
7,Galloway,Food Truck,Dive Bar,Mexican Restaurant,Business Service,American Restaurant,Public Art,Pub,Post Office,Pool,Plaza


In [104]:
#see if anywhere has a park as most common venue
wi_pk = wo_pz[wo_pz['1st Most Common Venue'] == 'Park']
wi_pk.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [105]:
#see if anywhere has a park as 2nd most common venue
wi_pk = wo_pz[wo_pz['2nd Most Common Venue'] == 'Park']
wi_pk.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [106]:
#see that Upper Arlington has a park as the 3rd most common venue.  They also don't have any pizza places in the top 10!
wi_pk = wo_pz[wo_pz['3rd Most Common Venue'] == 'Park']
wi_pk.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Upper Arlington,Gym,Playground,Park,American Restaurant,Neighborhood,Public Art,Pub,Post Office,Pool,Plaza


In [107]:
#create a UA df
ua = cbus[cbus['City'] == 'Upper Arlington']
ua.head()

,ZipCode,City,County,Latitude,Longitude
40,43221,Upper Arlington,Franklin,39.994508,-83.062408


In [108]:
#center on Upper Arlington
ua_latitude = cbus.loc[0, 'Latitude'] # City latitude value
ua_longitude = cbus.loc[0, 'Longitude'] # City longitude value

city_name = cbus.loc[40, 'City'] # City name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               cbus_latitude, 
                                                               cbus_longitude))

Latitude and longitude values of Upper Arlington are 40.076401, -83.1807987.


In [111]:
#pull all the nearby venues for Upper Arlington
ua_venues = getNearbyVenues(names=ua['City'],
                                   latitudes=ua['Latitude'],
                                   longitudes=ua['Longitude']
                                  )
print(ua_venues.shape)
ua_venues.head()

Upper Arlington
(53, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Upper Arlington,39.994508,-83.062408,Pinney Kelly Paper,39.999863,-83.062043,Gift Shop
1,Upper Arlington,39.994508,-83.062408,The Twisted Vine,39.989301,-83.055240,Wine Shop
2,Upper Arlington,39.994508,-83.062408,Point Tavern,39.995318,-83.072191,Dive Bar
3,Upper Arlington,39.994508,-83.062408,Chocolate Cafe,39.995871,-83.047340,Dessert Shop
4,Upper Arlington,39.994508,-83.062408,Bonifacio,39.991488,-83.049226,Filipino Restaurant


In [118]:
#there is one park in upper arlington
ua_pk = ua_venues[ua_venues['Venue Category'] == 'Park']
ua_pk.shape
ua_pk.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
46,Upper Arlington,39.994508,-83.062408,Falco Park,39.987349,-83.059395,Park


In [122]:
#plot out the location where they should build the new pizza place
map_ua = folium.Map(location=[ua_latitude, ua_longitude], zoom_start=10)

# add markers to map where they should build the new pizza place
for lat, lng, city in zip(ua_pk['Venue Latitude'], ua_pk['Venue Longitude'], ua_pk['City']):
    label = '{}, {}'.format(city, zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ua)  
    
map_ua

Results:

The result was that Upper Arlington was the best location to build in based on the low frequency of pizza places and high frequency of parks.

Discussion:

Based on the results, and since there was only a single park, I decided to plot the location of the park so that the new pizza place could be built next to the park.  That is the above map.  

Colclusion:

In conclusion, Upper Arlington was the best city for the new pizza place.  Building next to Falco Park would be ideal based on the pizza place's history with their previous Columbus locations.  